<div>
<img src='https://upload.wikimedia.org/wikipedia/commons/6/64/Sharif_University_Logo.jpg' alt="SUT logo" width=220 height=220 align=left class="saturate">

<br>
<font face="Times New Roman">
<div dir=ltr align=center>
<!-- <font color=0F5298 size=7> -->
<font color=0F5298 size=6>
    Introduction to Machine Learning <br> <br>
<!-- <font color=2565AE size=5> -->
<font size=5>
    Computer Engineering Department <br>
    Spring 2023 <br> <br>
<font color=606060 size=5>
    Project

____

### Full Name : Erfan Sadraiye
### Student Number : 99101835
### Colab Link:
___

In [ ]:
! pip install fasttext

Imports

In [ ]:
import nltk
import numpy as np
import pandas as pd
from tqdm import tqdm
from nltk import word_tokenize
import string
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import fasttext
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer
from time import time
from IPython.display import display

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import  classification_report


Download nltk files

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
ps = PorterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Reading data

In [ ]:
df = pd.read_csv('/content/data-train.csv')

Preprocess data
- tokenize
- lower case text
- remove punctuation
- stemming

In [ ]:
def preprocess_text(text, stopword_removal=True, lower_case=True,
                       punctuation_removal=True , porter_stemmer=ps):
    normalized_tokens = word_tokenize(text)
    if lower_case:
        normalized_tokens = [word.lower() for word in normalized_tokens]
    if punctuation_removal:
        normalized_tokens = [word for word in normalized_tokens if word not in string.punctuation]
    if stopword_removal:
        stopwords = [x.lower() for x in nltk.corpus.stopwords.words('english')]
        normalized_tokens = [word for word in normalized_tokens if word.lower() not in stopwords]
    normalized_tokens = [ps.stem(word) for word in normalized_tokens]
    return normalized_tokens


In [ ]:
df['preprocessed_phrases'] = df['Phrase'].apply(lambda x: preprocess_text(x))

split data to 3 data set
- Training 80%
- Validation 10%
- Test 10%

In [ ]:
train_df, remaining_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(remaining_df, test_size=0.5, random_state=42)

In [ ]:
train_phrases = train_df['preprocessed_phrases'].apply(lambda x: ' '.join(x))
val_phrases = val_df['preprocessed_phrases'].apply(lambda x: ' '.join(x))
test_phrases = test_df['preprocessed_phrases'].apply(lambda x: ' '.join(x))

y_train = train_df['Sentiment'].values
y_val = val_df['Sentiment'].values
y_test = test_df['Sentiment'].values

Fit TF-ITF on trainig phrases to convert each phrase to a vector

In [ ]:
tf_idf = TfidfVectorizer()
tf_idf.fit(train_phrases)


TfidfVectorizer()

In [ ]:
vectorized_train = tf_idf.transform(train_phrases).toarray()
vectorized_val = tf_idf.transform(val_phrases).toarray()
vectorized_test = tf_idf.transform(test_phrases).toarray()


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


Fit a Neural Network on our vectors

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm


class CustomDataset(Dataset):
    def __init__(self, x: list, y: list):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        return self.x[index], self.y[index]


class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(NeuralNetwork, self).__init__()

        layers = []
        prev_size = input_size

        for size in hidden_sizes:
            layers.append(nn.Linear(prev_size, size))
            layers.append(nn.BatchNorm1d(size))
            layers.append(nn.ReLU())
            prev_size = size

        layers.append(nn.Linear(prev_size, output_size))
        layers.append(nn.Softmax())

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        x = x.to(torch.float32)
        out = self.model(x)
        return out

class NeuralNetworkTrainer:
     def __init__(self, input_size, hidden_sizes, output_size, learning_rate=0.001, num_epochs=50):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.model = NeuralNetwork(input_size, hidden_sizes, output_size).to(self.device)
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)

        self.num_epochs = num_epochs

        self.best_model = None
        self.best_accuracy = 0.0


     def train_nn(self, x_train, y_train, x_val=None, y_val=None, batch_size=64):
        dataset = CustomDataset(x_train, y_train)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=1)

        if x_val is not None and y_val is not None:
            val_dataset = CustomDataset(x_val, y_val)
            val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=1)

        for epoch in range(self.num_epochs):
            # Training
            train_loss, train_accuracy = self.train_epoch(dataloader)

            # Validation
            if x_val is not None and y_val is not None:
                val_loss, val_accuracy = self.validate(val_dataloader)
                print(f"Epoch {epoch + 1}/{self.num_epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, "
                      f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

                if val_accuracy > self.best_accuracy:
                    self.best_accuracy = val_accuracy
                    self.best_model = self.model.state_dict().copy()
            else:
                print(f"Epoch {epoch + 1}/{self.num_epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")

        if self.best_model is not None:
            self.model.load_state_dict(self.best_model)


     def train_epoch(self, dataloader):
        self.model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        with tqdm(total=len(dataloader), ncols=80) as pbar:
            for inputs, labels in dataloader:
                inputs = inputs.to(self.device)
                labels = labels.to(self.device)

                self.optimizer.zero_grad()

                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)

                loss.backward()
                self.optimizer.step()

                running_loss += loss.item()

                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

                pbar.update(1)
                pbar.set_description(f"Loss: {loss.item():.4f}")

        epoch_loss = running_loss / len(dataloader)
        epoch_accuracy = correct / total

        return epoch_loss, epoch_accuracy

     def validate(self, dataloader):
        self.model.eval()
        running_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in dataloader:
                inputs = inputs.to(self.device)
                labels = labels.to(self.device)

                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)

                running_loss += loss.item()

                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

        epoch_loss = running_loss / len(dataloader)
        epoch_accuracy = correct / total

        return epoch_loss, epoch_accuracy

     def predict(self, x_test, batch_size=64):
        dataset = CustomDataset(x_test, [0] * len(x_test))
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=1)

        predictions = []

        self.model.eval()
        with torch.no_grad():
            for inputs, _ in dataloader:
                inputs = inputs.to(self.device)

                outputs = self.model(inputs)
                _, predicted = outputs.max(1)

                predictions.extend(predicted.cpu().numpy())

        return predictions





In [ ]:
input_size, hidden_sizes, output_size = len(vectorized_train[0]), [4096, 1024, 128], 5

nn_trainer_cv = NeuralNetworkTrainer(input_size, hidden_sizes, output_size,learning_rate=0.01,num_epochs=20)
nn_trainer_cv.train_nn(vectorized_train, y_train, vectorized_val, y_val)

  0%|                                                  | 0/2423 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
Loss: 1.2479: 100%|█████████████████████████| 2423/2423 [01:22<00:00, 29.32it/s]


Epoch 1/20, Train Loss: 1.3585, Train Accuracy: 0.5417, Validation Loss: 1.3038, Validation Accuracy: 0.5974


Loss: 1.3432: 100%|█████████████████████████| 2423/2423 [01:13<00:00, 33.19it/s]


Epoch 2/20, Train Loss: 1.3013, Train Accuracy: 0.6001, Validation Loss: 1.2740, Validation Accuracy: 0.6287


Loss: 1.3568: 100%|█████████████████████████| 2423/2423 [01:11<00:00, 33.73it/s]


...

Epoch 18/20, Train Loss: 1.2122, Train Accuracy: 0.6917, Validation Loss: 1.2029, Validation Accuracy: 0.7013


Loss: 1.3241: 100%|█████████████████████████| 2423/2423 [01:12<00:00, 33.63it/s]


Epoch 19/20, Train Loss: 1.2102, Train Accuracy: 0.6940, Validation Loss: 1.2016, Validation Accuracy: 0.7028


Loss: 1.1554: 100%|█████████████████████████| 2423/2423 [01:11<00:00, 33.68it/s]


Epoch 20/20, Train Loss: 1.2089, Train Accuracy: 0.6951, Validation Loss: 1.2004, Validation Accuracy: 0.7040


In [ ]:
preds = nn_trainer_cv.predict(vectorized_test)

print("Accuracy: ", accuracy_score(y_test, preds))
print("Recall Macro: ", recall_score(y_test, preds, average='macro'))
print("Recall Micro: ", recall_score(y_test, preds, average='micro'))
print("Precision Macro: ", precision_score(y_test, preds, average='macro'))
print("Precision Micro: ", precision_score(y_test, preds, average='micro'))
print("F1 Macro: ", f1_score(y_test, preds, average='macro'))
print("F1 Micro: ", f1_score(y_test, preds, average='micro'))
print(classification_report(y_test, preds))

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


              precision    recall  f1-score   support

           0       0.00      0.00      0.00       705
           1       0.64      0.59      0.62      2782
           2       0.75      0.91      0.82      7817
           3       0.63      0.67      0.65      3262
           4       0.00      0.00      0.00       939

    accuracy                           0.70     15505
   macro avg       0.40      0.43      0.42     15505
weighted avg       0.62      0.70      0.66     15505



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Failed Approachs

In [ ]:

clf = MultinomialNB()
clf.fit(vectorized_train, y_train)


MultinomialNB()

In [56]:
preds = clf.predict(vectorized_test)

In [57]:

print("Accuracy: ", accuracy_score(y_test, preds))
print("Recall Macro: ", recall_score(y_test, preds, average='macro'))
print("Recall Micro: ", recall_score(y_test, preds, average='micro'))
print("Precision Macro: ", precision_score(y_test, preds, average='macro'))
print("Precision Micro: ", precision_score(y_test, preds, average='micro'))
print("F1 Macro: ", f1_score(y_test, preds, average='macro'))
print("F1 Micro: ", f1_score(y_test, preds, average='micro'))
print(classification_report(y_test, preds))

Accuracy:  0.6259916156078684
Recall Macro:  0.42830763995924076
Recall Micro:  0.6259916156078684
Precision Macro:  0.5802413761272678
Precision Micro:  0.6259916156078684
F1 Macro:  0.4655752216953564
F1 Micro:  0.6259916156078684
              precision    recall  f1-score   support

           0       0.58      0.19      0.29       705
           1       0.54      0.36      0.43      2782
           2       0.67      0.87      0.75      7817
           3       0.55      0.47      0.51      3262
           4       0.57      0.25      0.34       939

    accuracy                           0.63     15505
   macro avg       0.58      0.43      0.47     15505
weighted avg       0.61      0.63      0.60     15505



In [50]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0).fit(vectorized_train, y_train)
y_pred_log = clf.predict(vectorized_test)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [53]:
print("Accuracy: ", accuracy_score(y_test, y_pred_log))
print("Recall Macro: ", recall_score(y_test, y_pred_log, average='macro'))
print("Recall Micro: ", recall_score(y_test, y_pred_log, average='micro'))
print("Precision Macro: ", precision_score(y_test, y_pred_log, average='macro'))
print("Precision Micro: ", precision_score(y_test, y_pred_log, average='micro'))
print("F1 Macro: ", f1_score(y_test, y_pred_log, average='macro'))
print("F1 Micro: ", f1_score(y_test, y_pred_log, average='micro'))
print(classification_report(y_test, y_pred_log))

Accuracy:  0.6259916156078684
Recall Macro:  0.42830763995924076
Recall Micro:  0.6259916156078684
Precision Macro:  0.5802413761272678
Precision Micro:  0.6259916156078684
F1 Macro:  0.4655752216953564
F1 Micro:  0.6259916156078684
              precision    recall  f1-score   support

           0       0.58      0.19      0.29       705
           1       0.54      0.36      0.43      2782
           2       0.67      0.87      0.75      7817
           3       0.55      0.47      0.51      3262
           4       0.57      0.25      0.34       939

    accuracy                           0.63     15505
   macro avg       0.58      0.43      0.47     15505
weighted avg       0.61      0.63      0.60     15505

Accuracy:  0.6259916156078684
Recall Macro:  0.42830763995924076
Recall Micro:  0.6259916156078684
Precision Macro:  0.5802413761272678
Precision Micro:  0.6259916156078684
F1 Macro:  0.4655752216953564
F1 Micro:  0.6259916156078684
              precision    recall  f1-score  

FastText

In [ ]:
import numpy as np
import fasttext

class FastTextSentenceVectorizer:

    def __init__(self, method='skipgram', model_path=None):
        self.method = method
        self.model = None
        self.model_path = model_path

    def train(self, texts):
        """
        train the fasttext model and save it into self.model
        Parameters
        ----------
        texts: list of list of str
        """
        file_name = 'data.txt'
        with open(file_name, 'w') as f:
            for item in texts:
                f.write("%s\n" % item)
            f.close()
        model = fasttext.train_unsupervised('data.txt', model=self.method)
        self.model = model


    def load_model(self):
        self.model = fasttext.load_model(self.model_path)

    def vectorize(self, sentences):
        if self.model is None:
            raise ValueError("FastText model is not trained or loaded. Please call 'train()' or 'load_model()' first.")

        vectors = []

        for sentence in sentences:
            vector = self.model.get_sentence_vector(sentence)
            vectors.append(vector)

        return np.vstack(vectors)


In [ ]:
ftv = FastTextSentenceVectorizer()
ftv.train(train_phrases.values)

In [ ]:
print(ftv.model.get_nearest_neighbors('fantastic'))
print(ftv.model.get_nearest_neighbors('awful'))

[(0.9083881974220276, 'fantasti'), (0.9080108404159546, 'fantas'), (0.904706597328186, 'fantasma'), (0.897767961025238, 'fantasia'), (0.8972718119621277, 'fantasi'), (0.8403797149658203, 'fantast'), (0.83005690574646, 'phantasm'), (0.8067594170570374, 'farcic'), (0.7783586978912354, 'ecstasi'), (0.7619239091873169, 'fanboy')]
[(0.8908090591430664, 'rueful'), (0.8890059590339661, 'cumul'), (0.8799886107444763, 'awe'), (0.8761646747589111, 'symbiot'), (0.874527633190155, 'true-to-lif'), (0.8683801889419556, 'pelosi'), (0.8664852380752563, 'larger-than-lif'), (0.8641816973686218, 'near-hypnot'), (0.8630688190460205, 'full-fledg'), (0.8617079854011536, 'abysm')]


In [ ]:
train_vectors = ftv.vectorize(train_phrases.values)
val_vectors = ftv.vectorize(val_phrases.values)
test_vectors = ftv.vectorize(test_phrases.values)
xxxx = ftv.vectorize(df['Phrase'].values)

In [ ]:
from sklearn.metrics import  classification_report

clf = LogisticRegression()
clf.fit(train_vectors, y_train)
preds = clf.predict(val_vectors)
print("Accuracy: ", accuracy_score(y_val, preds))
print("Recall Macro: ", recall_score(y_val, preds, average='macro'))
print("Recall Micro: ", recall_score(y_val, preds, average='micro'))
print("Precision Macro: ", precision_score(y_val, preds, average='macro'))
print("Precision Micro: ", precision_score(y_val, preds, average='micro'))
print("F1 Macro: ", f1_score(y_val, preds, average='macro'))
print("F1 Micro: ", f1_score(y_val, preds, average='micro'))
print(classification_report(y_val, preds))

Accuracy:  0.508287649145437
Recall Macro:  0.22753468942302804
Recall Micro:  0.508287649145437
Precision Macro:  0.406227455312127
Precision Micro:  0.508287649145437
F1 Macro:  0.2037122577775739
F1 Micro:  0.508287649145437
              precision    recall  f1-score   support

           0       0.57      0.01      0.01       680
           1       0.35      0.06      0.11      2680
           2       0.53      0.91      0.67      7917
           3       0.38      0.15      0.21      3295
           4       0.20      0.01      0.02       933

    accuracy                           0.51     15505
   macro avg       0.41      0.23      0.20     15505
weighted avg       0.45      0.51      0.41     15505



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
n_samples = 50000
rf_classifier.fit(train_vectors[:n_samples], y_train[:n_samples])

y_pred = rf_classifier.predict(test_vectors)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.50      0.19      0.28       705
           1       0.49      0.28      0.36      2782
           2       0.62      0.87      0.72      7817
           3       0.49      0.35      0.41      3262
           4       0.48      0.16      0.24       939

    accuracy                           0.58     15505
   macro avg       0.52      0.37      0.40     15505
weighted avg       0.55      0.58      0.54     15505



In [ ]:
input_size, hidden_sizes, output_size = len(train_vectors[0]), [150 , 50], 5
nn_trainer = NeuralNetworkTrainer(input_size, hidden_sizes, output_size,num_epochs=30)
nn_trainer.train_nn(train_vectors, y_train,val_vectors, y_val)


Loss: 1.2936: 100%|████████████████████████| 1939/1939 [00:14<00:00, 131.39it/s]


Epoch 1/30, Train Loss: 1.3497, Train Accuracy: 0.5485, Validation Loss: 1.3339, Validation Accuracy: 0.5606


Loss: 1.2936: 100%|████████████████████████| 1939/1939 [00:18<00:00, 105.68it/s]


Epoch 2/30, Train Loss: 1.3285, Train Accuracy: 0.5674, Validation Loss: 1.3310, Validation Accuracy: 0.5636


Loss: 1.2284: 100%|████████████████████████| 1939/1939 [00:14<00:00, 131.92it/s]


...

Epoch 28/30, Train Loss: 1.2685, Train Accuracy: 0.6334, Validation Loss: 1.3080, Validation Accuracy: 0.5917


Loss: 1.4044: 100%|████████████████████████| 1939/1939 [00:13<00:00, 141.12it/s]


Epoch 29/30, Train Loss: 1.2675, Train Accuracy: 0.6341, Validation Loss: 1.3101, Validation Accuracy: 0.5893


Loss: 1.0939: 100%|████████████████████████| 1939/1939 [00:13<00:00, 141.79it/s]


Epoch 30/30, Train Loss: 1.2676, Train Accuracy: 0.6335, Validation Loss: 1.3111, Validation Accuracy: 0.5890


In [ ]:
preds_ft = nn_trainer.predict(test_vectors)
print(classification_report(y_test, preds_ft))

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


              precision    recall  f1-score   support

           0       0.00      0.00      0.00       705
           1       0.45      0.43      0.44      2782
           2       0.68      0.80      0.73      7817
           3       0.45      0.49      0.47      3262
           4       0.00      0.00      0.00       939

    accuracy                           0.58     15505
   macro avg       0.31      0.34      0.33     15505
weighted avg       0.52      0.58      0.55     15505



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Kaggle

In [ ]:
test_df = pd.read_csv('pr-test-data.csv')
testt_phrases = test_df['Phrase'].apply(lambda x: preprocess_text(x))
testt_phrases = testt_phrases.apply(lambda x: ' '.join(x))
sentiment_nb = nn_trainer.predict(ftv.vectorize(testt_phrases))
test_df['Sentiment'] = sentiment_nb

ttttt = test_df.drop(['SentenceId','Phrase','PhraseId'],axis=1)
ttttt['ID'] = test_df.index + 1
ttttt.to_csv('nn_cv.csv',index=False)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
